In [ ]:
from numpynn import layers, networks, preprocessing, utils, optimizers, losses, metrics, activations, paddings
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import random

### Prepare data
Keras is used to fetch the data

In [ ]:
(x_train , y_train), (x_test , y_test) = keras.datasets.mnist.load_data()
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

Normalization

In [ ]:
x_train = x_train / 255
x_test = x_test / 255

Reshape the arrays to fit the models' dimension requirement

In [ ]:
x_train = preprocessing.expand_dims(x_train, 4)
y_train = preprocessing.expand_dims(y_train, 2)
x_test = preprocessing.expand_dims(x_test, 4)
y_test = preprocessing.expand_dims(y_test, 2)

print (f'{x_train.shape=}')
print (f'{y_train.shape=}')
print (f'{x_test.shape=}')
print (f'{y_test.shape=}')

### Build the neural network structre

In [ ]:
model = networks.FeedForward(input_shape=(28, 28, 1), layers=[
    layers.Convolution(nr_kernels=24, kernel_size=(5, 5), activation=activations.Relu()),
    layers.MaxPooling(pooling_window=(2,2)),
    layers.Flatten(),
    layers.Linear(nr_neurons=256, activation=activations.Tanh()),
    layers.Linear(nr_neurons=10, activation=activations.Softmax())
])

The network is compiled to internally connect it's layers and initialize the model.

In [ ]:
model.compile(
    optimizer=optimizers.adam(),
    loss_function=losses.crossentropy(),
    metric=metrics.Accuracy
)

In [ ]:
model.summary()

### Train the model

In [ ]:
model.train(x_train, y_train, epochs=1, batch_size=256)

In [ ]:
model.plot_training_loss()

In [ ]:
model.plot_neuron_activations()

In [ ]:
model.plot_neuron_gradients()

### Evaluate the model

In [ ]:
model.evaluate(x_test[:1000], y_test[:1000])

### Explore the inner workings
Pick a random image from the testing dataset.

In [ ]:
i = random.randint(0, len(x_test))
image = x_test[i]
image_tensor = np.expand_dims(x_test[i], 0)
plt.imshow(image, cmap='gray')

Use it to predict a number and show the probability distribution of the outcome.

In [ ]:
p = model.predict(image_tensor).flatten()
plt.bar(np.arange(0, 10), p.reshape(10,))

Every layer of the model can be accessed to explore their output. Here we iterate over all the kernels of the convolutional layer to explore what they learned to focus on in images.

In [ ]:
model.plot_conv_kernels()